In [1]:
import numpy as np 
from sklearn.model_selection import KFold
import pandas as pd
import sys
sys.path.append("../")
import gpflow
import tensorflow as tf
from gpsampling import GPActive
import xgboost
import multiprocessing

/home/nipun.batra/anaconda3/envs/gpflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nipun.batra/anaconda3/envs/gpflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nipun.batra/anaconda3/envs/gpflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nipun.batra/anaconda3/env

In [3]:
def active(train_days, i, j, isact):
    df = pd.read_csv("../../data/beijingb_scaled.csv", index_col = 0)
    df = df.rename(columns={'ts': 'Time', 'station_id': 'Station'})

    stations = df['Station'].unique()
    stations.sort()

    splits = 6

    kfout = KFold(n_splits=splits, shuffle=True, random_state=0)
    kfin = KFold(n_splits=splits - 1, shuffle=True, random_state=0)

    station_split = {}

    for kout, (sts_ftrain_index, sts_test_index) in enumerate(kfout.split(stations)):
        station_split[kout] = {}
        for kin, (sts_train_index, sts_val_index) in enumerate(kfin.split(sts_ftrain_index)):
            sts_test = stations[sts_test_index]
            sts_val = stations[sts_ftrain_index[sts_val_index]]
            sts_train = stations[sts_ftrain_index[sts_train_index]]
            station_split[kout][kin] = {
                'test': sts_test,
                'train': sts_val,
                'pool': sts_train,
            }

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)


    gp = GPActive(
                    df = df,
                    train_stations = list(station_split[i][j]['train']),
                    pool_stations = list(station_split[i][j]['pool']),
                    test_stations = list(station_split[i][j]['test']),
                    context_days = train_days - 1 , # train_days - 1
                    frequency = 30, # 
                    test_days = 365 - train_days,  # 
                    train_days = train_days, # 10, 20, 30
                    number_to_query = 1,
                    number_of_seeds= 5,
                    fname=[i, j]
                )
    print(isact)
    if isact == 1:
        gp.active_gp()
    else:
        gp.random_sampling()

In [4]:
active(10, 0, 0, 1)

INIT DONE
True True



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 249.240687
  Number of iterations: 114
  Number of functions evaluations: 140


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 249.240687
  Number of iterations: 114
  Number of functions evaluations: 140


...

Current Day before update: 355

Current Day after update: 356
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 760.891509
  Number of iterations: 272
  Number of functions evaluations: 332


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 760.891509
  Number of iterations: 272
  Number of functions evaluations: 332


Current Day before update: 356

Current Day after update: 357
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 765.547757
  Number of iterations: 183
  Number of functions evaluations: 226


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 765.547757
  Number of iterations: 183
  Number of functions evaluations: 226


Current Day before update: 357

Current Day after update: 358
index 358 is out of bounds for axis 0 with size 358
